# Here Importing the Eseantial libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
import os
from tensorflow import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
from keras import applications
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

# Load the Lables of the images from train.csv file 

In [ ]:
dataframe = pd.read_csv('/kaggle/input/human-protein-atlas-image-classification/train.csv')
dataframe.head(5)

 * Defining the input shape of 1st input layer 
 * decalare the batch size
 * save the path of train images in the variable 'path_to_train'

In [ ]:
INPUT_SHAPE = (512, 512, 3)
BATCH_SIZE = 16
path_to_train = '/kaggle/input/human-protein-atlas-image-classification/train/'

 Adding the column with the name of complete_path and load the full path of each image on csv 

In [ ]:
dataframe["complete_path"] = path_to_train + dataframe["Id"]
dataframe.head(5)

#  Visualizing the pictures



In [ ]:
import random
fig, axes = plt.subplots(3, 4, figsize=(10, 10))
for i in range(3):
    for j in range(4):
        idx = random.randint(0, dataframe.shape[0])
        row = dataframe.iloc[idx,:]
        path = row.complete_path
        red = np.array(Image.open(path + '_red.png'))
        green = np.array(Image.open(path + '_green.png'))
        blue = np.array(Image.open(path + '_blue.png'))
        im = np.stack((
                red,
                green,
                blue),-1)
        axes[i][j].imshow(im)
        axes[i][j].set_title(row.Target)
        axes[i][j].set_xticks([])
        axes[i][j].set_yticks([])
fig.tight_layout()
fig.show(5);

Spliting the data into parts train and val(valiation)

In [ ]:
train, val = train_test_split(dataframe, test_size=0.2, random_state=42)


In [ ]:
print(f'Shape of train: {train.shape}')
print(f'Shape of val: {val.shape}')

Cleaning the data for better results 

In [ ]:
def get_clean_data(df):
    targets = []
    paths = []
    for _, row in df.iterrows():
        target_np = np.zeros((28))
        t = [int(t) for t in row.Target.split()]
        target_np[t] = 1
        targets.append(target_np)
        paths.append(row.complete_path)
    return np.array(paths), np.array(targets)

In [ ]:
train_path, train_target = get_clean_data(train)
val_path, val_target = get_clean_data(val)

printing train and val path and target



In [ ]:
print(f'Train path shape: {train_path.shape}')
print(f'Train target shape: {train_target.shape}')
print(f'Val path shape: {val_path.shape}')
print(f'Val target shape: {val_target.shape}')


creating datasets from cleaned data


In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_path, train_target))
val_data = tf.data.Dataset.from_tensor_slices((val_path, val_target))

Experimental API for building input pipelines.



In [ ]:
def load_data(path, target):
    red = tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_red.png'), channels=1), [2])
    blue = tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_blue.png'), channels=1), [2])
    green = tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_green.png'), channels=1), [2])
    #yellow=tf.squeeze(tf.image.decode_png(tf.io.read_file(path+'_yellow.png'), channels=1), [2])
    img = tf.stack((
                red,
                green,
                blue), axis=2)
    return img, target

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_data = train_data.map(load_data, num_parallel_calls=AUTOTUNE)
val_data = val_data.map(load_data, num_parallel_calls=AUTOTUNE)

Adjust the contrast of an image or images by a random factor.
Apply Data Augmentation



In [ ]:
def image_augment(img, target):
    img = tf.image.random_contrast(img, lower=0.3, upper=2.0)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, max_delta=0.1)
    return img, target
    
train_data = train_data.map(image_augment, num_parallel_calls=AUTOTUNE)

Better performance with the tf.data API


In [ ]:
train_data_batches = train_data.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
val_data_batches = val_data.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

# Building the Model Architecture 

Using transfer learning and fine tuning on InceptionResNetV2

In [ ]:
inception_model = applications.InceptionResNetV2(include_top=False, weights='imagenet')

inception_model.trainable = False

input_layer = Input(shape=INPUT_SHAPE)
x = inception_model(input_layer)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(28, activation='softmax')(x)
model = Model(input_layer, output)

model.summary()

model compilation 

In [ ]:
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

Now finally fit the model for trianing ...

In [ ]:
history = model.fit(train_data_batches, validation_data = val_data_batches,steps_per_epoch = 50, epochs=10)

Visualizing the Results 

In [ ]:
model.save('InceptionV2.h5')

In [ ]:
binary_accuracy=history.history['accuracy']
val_binary_accuracy=history.history['val_accuracy']
epochs=range(1,len(binary_accuracy)+1)
plt.plot(epochs,binary_accuracy,'b',label='Training accuracy')  
plt.plot(epochs,val_binary_accuracy,'r',label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()
plt.show()

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(binary_accuracy)+1)
plt.plot(epochs,loss,'b',label='Training loss')
plt.plot(epochs,val_loss,'r',label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.figure()
plt.show()

In [ ]:
dataframe = pd.read_csv('/kaggle/input/human-protein-atlas-image-classification/sample_submission.csv')
dataframe.head(5)

In [ ]:
INPUT_SHAPE = (512, 512, 3)
BATCH_SIZE = 16
path_to_test = '/kaggle/input/human-protein-atlas-image-classification/test/'

In [ ]:
dataframe["complete_path"] = path_to_test + dataframe["Id"]
dataframe.head(5)

In [ ]:
targets = dataframe['Predicted']
test_path ,targets = np.array(path_to_test), np.array(targets)

In [ ]:
savedModel = load_model('./kaggle/output/hpa-new/InceptionV2.h5')